In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from datetime import datetime, timedelta

# initiating SparkSession
spark = SparkSession.builder \
    .appName("Delta Lake Exercise") \
    .getOrCreate()



The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/Users/vishnureddy/opt/anaconda3/lib/python3.9/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [ ]:
# STAGE 1

# read source json file and create df
df = spark.read.json("dbfs:nbcdotcom/tve-de-interview-repo/1970movies.json")

# flattening the df
df_flat = df.select(
    col("title"),
    col("year"),
    col("cast"),
    col("genres"),
    col("href"),
    col("extract"),
    col("thumbnail"),
    col("thumbnail_width"),
    col("thumbnail_height")
)

# write to Delta table partitioned by year
df_flat.write.format("delta").partitionBy("year").save("dbfs:nbcdotcom/tve-de-interview-repo/delta/1970movies_source")

# I'm unclear of the requirement in Step 4 to add adhoc paragraph however I'm attempting based on my understanding.
# Restoring 7 days back point in time data from Delta Lake using timetravel (Assuming a delta table retention policy is set to > 7 days)

timestamp_seven_days_ago = datetime.now() - timedelta(days=7)
delta_df_7days_ago = spark.read.format("delta").option("timestampAsOf", timestamp_seven_days_ago.strftime("%Y-%m-%d %H:%M:%S")).load("dbfs:nbcdotcom/tve-de-interview-repo/delta/1970movies_source")




In [ ]:
# STAGE 2

from pyspark.sql.functions import monotonically_increasing_id

# select distinct movie titles
distinct_movies = delta_df_7days_ago.select("title").distinct()

# assign unique movieId values for each movie title
movie_dimension = distinct_movies.withColumn("movieId", monotonically_increasing_id())

# udate delta table created in Stage 1 with movieId corresponding to title
delta_df_with_movie_id = delta_df_7days_ago.join(movie_dim_df, on="title", how="left_outer").select(
    col("movieId"),
    col("title"),
    col("year"),
    col("cast"),
    col("genres"),
    col("href"),
    col("extract"),
    col("thumbnail"),
    col("thumbnail_width"),
    col("thumbnail_height")
)

# updating source delta table created in stage1 with df to include movieid field
delta_df_with_movie_id.write.format("delta").partitionBy("year").mode("overwrite").option("mergeSchema", "true").save("dbfs:nbcdotcom/tve-de-interview-repo/delta/1970movies_source")

# Stop SparkSession
spark.stop()
